In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, svm,  tree, preprocessing, metrics
import sklearn.ensemble as ske
import tensorflow as tf
from tensorflow.contrib import skflow

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
#load data as dataframes
test_data = pd.read_csv('./test.csv',index_col=False)
train_data = pd.read_csv ('./train.csv', index_col=False)

In [5]:
def drop_and_purge(table,colums_to_drop,purge=True):
    for col in colums_to_drop:
        try:
            del table[col]
        except KeyError as exp:
            print('colum',exp,'not found in table')
    if purge:
        table.dropna()

In [6]:
def quantise(table,dict_to_quantize):
    '''takes table and dict {column_name:[{value_in_table:value_to_replace_with}]}
    example:- quantise(table,{Sex:[{male:1},{female:0}]}) will replace male and female in Sex column of table with 1 and 0'''
    for val in dict_to_quantize:
        for string_to_replace in dict_to_quantize[val]:
            value_to_replace_with = dict_to_quantize[val][string_to_replace]
            table[val].replace(string_to_replace,value_to_replace_with,inplace=True)

In [8]:
#test drop_and_purge functions
coloums_to_drop = ['Name','Ticket','Cabin','Fare']
drop_and_purge(test_data,coloums_to_drop,purge=False)
drop_and_purge(train_data,coloums_to_drop)

colum 'Name' not found in table
colum 'Ticket' not found in table
colum 'Cabin' not found in table
colum 'Fare' not found in table
colum 'Name' not found in table
colum 'Ticket' not found in table
colum 'Cabin' not found in table
colum 'Fare' not found in table


In [32]:
quantise(test_data,{'Sex':{'male':1,'female':0}})
quantise(test_data,{'Embarked':{'Q':0,'C':1,'S':2}})
quantise(train_data,{'Sex':{'male':1,'female':0}})
quantise(train_data,{'Embarked':{'Q':0,'C':1,'S':2}})
print(test_data.head())
print(train_data.head())

   PassengerId  Pclass  Sex   Age  SibSp  Parch  Embarked  Survived
0          892       3    1  34.5      0      0         0       0.0
1          893       3    0  47.0      1      0         2       0.0
2          894       2    1  62.0      0      0         0       1.0
3          895       3    1  27.0      0      0         2       0.0
4          896       3    0  22.0      1      1         2       0.0
   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch  Embarked
0            1         0       3    1  22.0      1      0       2.0
1            2         1       1    0  38.0      1      0       1.0
2            3         1       3    0  26.0      0      0       2.0
3            4         1       1    0  35.0      1      0       2.0
4            5         0       3    1  35.0      0      0       2.0


In [34]:
data_dist_sex = train_data['Sex'].value_counts()
print(data_dist_sex[1])


# In[10]:

male_survived = 0
female_survived = 0
for index, row in train_data.iterrows():
    if row['Survived'] == 1:
        if row['Sex'] == 1:
            male_survived = male_survived + 1
        else:
            female_survived = female_survived + 1
print(male_survived,female_survived)


# In[11]:

def survivedCalc(prob):
    random = np.random.choice([0, 1], p=[1-prob, prob])
    #print(random)
    return random


# In[12]:

perct_male_survived = male_survived/data_dist_sex[1]
perct_female_survived = female_survived/data_dist_sex[0]

453
93 195


In [22]:
clf_dt = tree.DecisionTreeClassifier(max_depth=10)
train_data = train_data.dropna()
Y = train_data['Survived'].values
X = train_data.drop(['Survived','PassengerId'], axis=1).values

print(X)
print(Y)

clf_dt.fit (X, Y)
clf_dt.score(X,Y)

[[  3.   1.  22.   1.   0.   2.]
 [  1.   0.  38.   1.   0.   1.]
 [  3.   0.  26.   0.   0.   2.]
 ..., 
 [  1.   0.  19.   0.   0.   2.]
 [  1.   1.  26.   0.   0.   1.]
 [  3.   1.  32.   0.   0.   0.]]
[0 1 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0
 0 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0
 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1
 0 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 1
 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0
 0 1 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1
 1 1 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0
 0 0 0 1 0 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 1
 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 1 0 0 1 1 0 1 1 0 0
 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0

0.9058988764044944

In [35]:
for index, row in test_data.iterrows():
    if row.isnull().values.any() == False:
        data = row[[ 'Pclass' , 'Sex','Age','SibSp','Parch','Embarked']]
        data = data.values
        val = clf_dt.predict(data.reshape(-1,6))
        test_data = test_data.set_value(index,'Survived',int(val))
    else:
        if row['Sex'] == 'male':
            val = survivedCalc(perct_male_survived)
            test_data = test_data.set_value(index,'Survived',val)
        else:
            val = survivedCalc(perct_female_survived)
            test_data = test_data.set_value(index,'Survived',val)

In [36]:
gensubmission = test_data[['PassengerId', 'Survived']].copy()
gensubmission.Survived = gensubmission.Survived.astype(int)
gensubmission.to_csv('./DSsubmission.csv',index=False)

In [37]:
def custom_model(X, y):
    layers = skflow.ops.dnn(X, [20, 40, 20], tf.tanh)
    return skflow.models.logistic_regression(layers, y)

In [38]:
tf_clf_c = skflow.TensorFlowEstimator(model_fn=custom_model, n_classes=2, batch_size=256, steps=1000, learning_rate=0.05)
tf_clf_c.fit(X, y)

NameError: name 'skflow' is not defined